In [2]:
!pip install tqdm

In [1]:
import pandas as pd
import numpy as np
import datetime
from shapely.geometry import Point
# from tqdm import tqdm
import time
import os
# import matplotlib.pyplot as plt
import geopandas as gpd
from shapely.geometry.point import Point
import geopandas as gpd
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# reading shapefile of uttarakhand districts
shape_file_path = r'C:\Users\User\Downloads\Mobility_traces_new_data_code\GeoJSON files\Uttarakhand\UK_Districts.geojson'
poly_file = gpd.read_file(shape_file_path)
shape_uttarakhand = poly_file["geometry"]
# shape_uttarakhand
# poly_file

In [8]:
# df_useful = pd.read_csv(r'C:\Users\User\Downloads\Mobility_traces_new_data_code\all visitors\visitors_with_all_states_and_districts(without_NaN).csv')
df_useful[df_useful['district']=='Outside']['state'].unique()

array(['Himachal Pradesh', 'Uttar Pradesh', 'NCT of Delhi', 'Chandigarh',
       'Maharashtra', 'Punjab', 'Haryana', 'Gujarat', 'West Bengal',
       'Jharkhand', 'Bihar', 'Telangana', 'Karnataka', 'Rajasthan',
       'Jammu & Kashmir', 'Madhya Pradesh', 'Assam', 'Andhra Pradesh',
       'Chhattisgarh', 'Odisha', 'Kerala', 'Tamil Nadu', 'Daman & Diu',
       'Sikkim', 'Nagaland', 'Tripura', 'Meghalaya', 'Goa',
       'Dadara & Nagar Havelli', 'Manipur', 'Mizoram',
       'Arunanchal Pradesh', 'Andaman & Nicobar Island', 'Puducherry'],
      dtype=object)

In [14]:
# function to stamp dictricts againt lat_log_coordinates
def add_districts(df, poly_file):
  geometry = [Point(xy) for xy in zip(df.longitude, df.latitude)]
  coordinates_gdf = gpd.GeoDataFrame(df, geometry=geometry)
  joined_gdf = gpd.sjoin(coordinates_gdf, poly_file, how='left', predicate='within')
  joined_gdf = joined_gdf[['maid', 'datetime', 'latitude', 'longitude', 'name','index_right']]
  joined_gdf.rename(columns={'index_right': 'district_id', 'name': 'district'}, inplace=True)
  # Replace NaN values in multiple columns
  joined_gdf.fillna({'district_id': 13, 'district': 'Outside'}, inplace=True)
  joined_gdf.reset_index(drop=True, inplace=True)
  return joined_gdf

In [15]:
df_useful_with_districts= add_districts(df_useful, poly_file=poly_file)

In [22]:
after_rec = len(df_useful_with_districts)
before_rec = len(df_useful)
print(f'df_useful: {before_rec}  df_useful_with_districts: {after_rec} \nDifference: {before_rec-after_rec}')

df_useful: 4554069  df_useful_with_districts: 4554236 
Difference: -167


In [58]:
import geopandas as gpd
from shapely.geometry import Point

# Load the polygon data into a GeoDataFrame (assuming poly_file is already loaded)
# poly_file = gpd.read_file('path_to_polygon_file.geojson')

# Step 1: Identify Overlapping Polygons

# Create a spatial index
sindex = poly_file.sindex

# Initialize a list to hold overlapping pairs
overlapping_pairs = []

# Iterate through each polygon and check for overlaps
for idx, polygon in poly_file.iterrows():
    possible_matches_index = list(sindex.intersection(polygon.geometry.bounds))
    possible_matches = poly_file.iloc[possible_matches_index]
    for match_idx, match_polygon in possible_matches.iterrows():
        if polygon.geometry.intersects(match_polygon.geometry) and idx != match_idx:
            overlapping_pairs.append((idx, match_idx))

# Print the number of overlapping pairs of polygons
print(f'Number of overlapping pairs of polygons: {len(overlapping_pairs)}')

# Optional: Print the actual overlapping pairs for further investigation
print("Overlapping pairs of polygon indices:")
print(overlapping_pairs)

# Step 2: Check for Multiple Matches in Spatial Join

# def add_districts(df, poly_file):
#     # Create a geometry column from latitude and longitude
#     geometry = [Point(xy) for xy in zip(df.longitude, df.latitude)]
#     coordinates_gdf = gpd.GeoDataFrame(df, geometry=geometry)
    
#     # Perform spatial join with the predicate 'within'
#     joined_gdf = gpd.sjoin(coordinates_gdf, poly_file, how='left', predicate='within')
    
#     # Check for multiple matches
#     match_counts = joined_gdf.groupby(['maid', 'datetime', 'latitude', 'longitude']).size()
#     multiple_matches = match_counts[match_counts > 1]
    
#     if not multiple_matches.empty:
#         print('Points with multiple matches:')
#         print(multiple_matches)
    
#     # Select relevant columns and rename them
#     joined_gdf = joined_gdf[['maid', 'datetime', 'latitude', 'longitude', 'name', 'index_right']]
#     joined_gdf.rename(columns={'index_right': 'district_id', 'name': 'district'}, inplace=True)
    
#     # Fill NaN values for 'district_id' and 'district'
#     joined_gdf.fillna({'district_id': 13, 'district': 'Outside'}, inplace=True)
    
#     # Check for duplicates and remove them if necessary
#     joined_gdf.drop_duplicates(subset=['maid', 'datetime', 'latitude', 'longitude'], inplace=True)
    
#     # Reset the index
#     joined_gdf.reset_index(drop=True, inplace=True)
    
#     return joined_gdf

# # Use the function on your data
# df_useful_with_districts = add_districts(df_useful, poly_file=poly_file)


Number of overlapping pairs of polygons: 54
Overlapping pairs of polygon indices:
[(0, 5), (0, 6), (0, 4), (0, 3), (1, 3), (1, 2), (1, 7), (1, 12), (2, 3), (2, 1), (3, 0), (3, 6), (3, 11), (3, 2), (3, 7), (3, 8), (3, 1), (4, 5), (4, 0), (5, 0), (5, 6), (5, 10), (5, 4), (6, 5), (6, 0), (6, 9), (6, 10), (6, 11), (6, 3), (7, 3), (7, 8), (7, 1), (7, 12), (8, 11), (8, 3), (8, 7), (8, 12), (9, 6), (9, 10), (9, 11), (10, 5), (10, 6), (10, 9), (10, 11), (11, 6), (11, 9), (11, 10), (11, 3), (11, 8), (11, 12), (12, 11), (12, 7), (12, 8), (12, 1)]


In [62]:
poly_file.to_csv(r'C:\Users\User\Downloads\Mobility_traces_new_data_code\GeoJSON files\polyfile.csv', index=False)

In [ ]:
# Calculate counts of maids in both dataframes
count_A = df_useful['maid'].value_counts()
count_B = df_useful_with_districts['maid'].value_counts()
count_A.sort_values()
count_B.sort_values()

# Ensure both Series have the same index labels
merged_counts = pd.concat([count_A, count_B], axis=1, keys=['count_A', 'count_B']).fillna(0)

# Find maids where the count does not match between df_A and df_B
mismatched_maids = merged_counts[merged_counts['count_A'] != merged_counts['count_B']]

# Print or further process the mismatched maids
print("Maids with count mismatches between A and B:")
print(mismatched_maids)
# Find maids where the count does not match between df_A and df_B
# mismatched_maids = count_A[count_A != count_B]
print(len(count_A))
print(len(count_B))
# mismatched_maids

In [40]:
# print(count_A.loc['5fe44f45-ecda-4702-97e8-0a73607e8aab'])  #168
# print(count_B.loc['5fe44f45-ecda-4702-97e8-0a73607e8aab'])  #175
# count_A


df_useful_with_districts.to_csv(r'C:\Users\User\Downloads\Mobility_traces_new_data_code\records_with_districts\df_useful_with_districts.csv')
mismatched_maids.to_csv(r'C:\Users\User\Downloads\Mobility_traces_new_data_code\records_with_districts\mismatched_maids.csv')

In [43]:
# mismatched_maids['count_A'].sum() - mismatched_maids['count_B'].sum()
mismatch_maids = mismatched_maids.index.tolist()
len(mismatch_maids)

48

In [44]:
mismatching_maids_df_useful = df_useful[df_useful['maid'].isin(set(mismatch_maids))]
mismatching_maids_df_useful_with_districts = df_useful_with_districts[df_useful_with_districts['maid'].isin(set(mismatch_maids))]
mismatching_maids_df_useful_with_districts.to_csv(r'C:\Users\User\Downloads\Mobility_traces_new_data_code\records_with_districts\comparison of mismatching maids\mismatching_maids_df_useful_with_districts.csv')
mismatching_maids_df_useful.to_csv(r'C:\Users\User\Downloads\Mobility_traces_new_data_code\records_with_districts\comparison of mismatching maids\mismatching_maids_df_useful.csv')


In [48]:
pip install rtree

You should consider upgrading via the 'c:\Users\Admin\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [49]:
import geopandas as gpd
from shapely.geometry import Polygon
from rtree import index

# Assuming poly_file is a GeoDataFrame with polygons
# Create spatial index
sindex = poly_file.sindex

# List to store overlapping pair indices
overlapping_pairs = []

# Iterate through polygons in poly_file
for idx, polygon in poly_file.iterrows():
    # Find possible matches using spatial index
    possible_matches_index = list(sindex.intersection(polygon.geometry.bounds))
    possible_matches = poly_file.iloc[possible_matches_index]
    
    # Iterate through possible matches
    for match_idx, match_polygon in possible_matches.iterrows():
        if polygon.geometry.intersects(match_polygon.geometry) and idx != match_idx:
            overlapping_pairs.append((idx, match_idx))

# Create a set of unique overlapping pairs to avoid duplicates
overlapping_pairs = set(tuple(sorted(pair)) for pair in overlapping_pairs)

# Convert set back to list for printing or further use
overlapping_pairs = list(overlapping_pairs)

# Remove overlapping pairs from poly_file
indices_to_remove = set()
for pair in overlapping_pairs:
    indices_to_remove.update(pair)

# Create a new DataFrame excluding overlapping pairs
poly_file_cleaned = poly_file.drop(index=indices_to_remove)

# Print the number of overlapping pairs of polygons found
print(f'Number of overlapping pairs of polygons: {len(overlapping_pairs)}')

# Optional: Print the actual overlapping pairs for further investigation
print("Overlapping pairs of polygon indices:")
print(overlapping_pairs)

Number of overlapping pairs of polygons: 27
Overlapping pairs of polygon indices:
[(3, 7), (5, 10), (0, 5), (8, 12), (9, 11), (1, 3), (1, 12), (6, 11), (4, 5), (5, 6), (3, 6), (8, 11), (9, 10), (1, 2), (0, 4), (10, 11), (7, 12), (6, 10), (3, 11), (3, 8), (0, 3), (0, 6), (2, 3), (11, 12), (1, 7), (6, 9), (7, 8)]


In [115]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point

# Function to stamp districts against lat_lon_coordinates
def add_districts_1(df, poly_file):
    geometry = [Point(xy) for xy in zip(df.longitude, df.latitude)]
    coordinates_gdf = gpd.GeoDataFrame(df, geometry=geometry)
    print(f' coordinates_gdf shape: {coordinates_gdf.shape}')
    
    # Perform the spatial join
    joined_gdf = gpd.sjoin(coordinates_gdf, poly_file, how='left', predicate='within')
    print(f' joined_gdf shape: {joined_gdf.shape}')
    
    # Select relevant columns and rename them
    joined_gdf = joined_gdf[['maid', 'datetime', 'latitude', 'longitude', 'name', 'index_right']]
    joined_gdf.rename(columns={'index_right': 'district_id', 'name': 'district'}, inplace=True)
    
    # Replace NaN values in district_id and district columns
    joined_gdf.fillna({'district_id': 13, 'district': 'Outside'}, inplace=True)
    print(f' joined_gdf shape after filling NaN: {joined_gdf.shape}')
    # Group by datetime and ensure only one entry per latitude-longitude pair per datetime
    grouped_gdf = joined_gdf.groupby(['maid','datetime', 'latitude', 'longitude']).first().reset_index()
    print(f' grouped_gdf shape: {grouped_gdf.shape}')
    # return grouped_gdf
    return grouped_gdf[['maid', 'datetime', 'latitude','longitude', 'district', 'district_id']]

In [ ]:
df_grouped_gdf = add_districts_1(df, poly_file)
df_grouped_gdf

In [ ]:
df

In [ ]:
import pandas as pd


df = pd.read_csv(r'C:\Users\User\Downloads\Mobility_traces_new_data_code\records_with_districts\comparison of mismatching maids\rec_with_different_dist_same_lat_long.csv')
# Sort the dataframe
df.sort_values(by=['maid', 'datetime', 'latitude', 'longitude'], inplace=True)

# # Add a new column 'prev_dist_id' to store the previous district ID
# df['prev_dist_id'] = df['district_id'].shift()

# # Function to determine if a record should be kept
# def keep_record(row):
#     if pd.isna(row['prev_dist_id']):
#         return True
#     if row['district_id'] != row['prev_dist_id']:
#         prev_row = df.loc[row.name - 1]
#         if row['latitude'] == prev_row['latitude'] and row['longitude'] == prev_row['longitude']:
#             return False
#     return True

# Apply the function to filter the dataframe
df_filtered = df[df.apply(keep_record, axis=1)]
df_filtered=df_filtered.reset_index()

# print(df_filtered)
df_filtered


In [ ]:
df=df.reset_index()
df

In [116]:
df_useful_with_districts_cleaned= add_districts_1(df_useful, poly_file)

 coordinates_gdf shape: (4554069, 7)
 joined_gdf shape: (4554236, 14)
 joined_gdf shape after filling NaN: (4554236, 6)
 grouped_gdf shape: (3729555, 6)


In [117]:
print(df_useful_with_districts_cleaned.shape)
print(df_useful.shape)

(3729555, 6)
(4554069, 6)


In [108]:
# poly_file_cleaned.to_csv(r'C:\Users\User\Downloads\Mobility_traces_new_data_code\GeoJSON files\poly_file_cleaned.csv')

In [120]:
df_useful_with_districts_cleaned.to_csv(r'C:\Users\User\Downloads\Mobility_traces_new_data_code\records_with_districts\df_useful_with_districts_cleaned_latest.csv')

In [123]:
sample_maids= df_useful['maid'].sample(n=5, random_state=45).tolist()
sample_maids

['cf9552e1-d39c-48c5-98fb-b17771952e02',
 'c711fcc4-5ce9-4839-9590-3d3e639d263b',
 '44577152-2147-4cf7-a2d1-7ca70d39f64a',
 '9fac1085-aaf8-435c-912e-292978a19cf7',
 '4d828342-e350-4c9e-a554-d762f1b107ed']

In [126]:
sample_df = df_useful[df_useful['maid'].isin(set(sample_maids))]
sample_df_with_districts = df_useful_with_districts_cleaned[df_useful_with_districts_cleaned['maid'].isin(set(sample_maids))]

In [127]:
sample_df.to_csv(r'C:\Users\User\Downloads\Mobility_traces_new_data_code\records_with_districts\sample id traces\sample_df.csv', index=False)
sample_df_with_districts.to_csv(r'C:\Users\User\Downloads\Mobility_traces_new_data_code\records_with_districts\sample id traces\sample_df_with_districts.csv', index=False)

In [57]:
# mismatching_maids_df_useful_with_districts_cleaned = df_useful_with_districts_cleaned[df_useful_with_districts_cleaned['maid'].isin(mismatch_maids)]
# mismatching_maids_df_useful_with_districts_cleaned.to_csv(r'C:\Users\User\Downloads\Mobility_traces_new_data_code\records_with_districts\comparison of mismatching maids\mismatching_maids_df_useful_with_districts_cleaned.csv')

Computing OD Matrix

In [ ]:
df_useful_with_districts_cleaned.groupby(['district_id', 'district']).size().reset_index()#.iloc[:, [8, 9]] #.first().reset_index()

In [10]:
from datetime import datetime
import pandas as pd

def summarize_district_travel(visitors, start_date, end_date):
    """
    Summarizes the travel between districts within a given date range.

    Parameters:
    visitors (pd.DataFrame): The dataframe containing visitor records.
    start_date (str): The start date in the format 'YYYY-MM-DD'.
    end_date (str): The end date in the format 'YYYY-MM-DD'.

    Returns:
    pd.DataFrame: A summary dataframe showing the count of people traveling from one district to another.
    """
    # Ensure datetime column is in datetime format
    visitors['datetime'] = pd.to_datetime(visitors['datetime'], format='%Y-%m-%d %H:%M:%S')

    # Extract date part
    visitors['date'] = visitors['datetime'].dt.date

    # Convert start_date and end_date to datetime objects
    start_date = datetime.strptime(start_date, '%Y-%m-%d').date()
    end_date = datetime.strptime(end_date, '%Y-%m-%d').date()

    # Filter the dataframe based on the date range
    mask = (visitors['date'] >= start_date) & (visitors['date'] <= end_date)
    filtered_visitors = visitors.loc[mask].copy()

    if filtered_visitors.empty:
        print("No records found within the given date range.")
        return pd.DataFrame(columns=['Travelled from', 'Travelled to', 'Count of people'])

    # Shift the district information to compare with the previous row
    # filtered_visitors['prev_district'] = filtered_visitors.groupby('maid')['district_id'].shift(1)
    filtered_visitors['prev_district_name'] = filtered_visitors.groupby('maid')['district'].shift(1)

    # Identify changes in district
    filtered_visitors['changed_district'] = filtered_visitors['district'] != filtered_visitors['prev_district_name']

    # Filter only the rows where district has changed and the previous district is not NaN
    visitors_changes = filtered_visitors[filtered_visitors['changed_district'] & filtered_visitors['prev_district_name'].notna()]

    if visitors_changes.empty:
        print("No district changes found within the given date range.")
        return pd.DataFrame(columns=['Travelled from', 'Travelled to', 'Count of people'])

    # Group by 'Travelled from' and 'Travelled to' and count number of transitions
    travel_summary = visitors_changes.groupby(['prev_district_name', 'district']).size().reset_index(name='Count')

    # Rename columns for clarity
    travel_summary.columns = ['Travelled from', 'Travelled to', 'Count of travels']
    # Create OD matrix
    od_matrix = pd.pivot_table(travel_summary, values='Count of travels', index='Travelled from', columns='Travelled to', fill_value=0)

    print("Origin-Destination Matrix:")
    print(od_matrix)

    return od_matrix

# print(OD_matrix)


In [21]:
from datetime import datetime, timedelta

# Define the date range and call the function for each date, saving the result as a CSV file
start_date = datetime.strptime('2022-09-03', '%Y-%m-%d')
end_date = datetime.strptime('2022-09-14', '%Y-%m-%d')
current_date = start_date
while current_date <= end_date:
    # Generate OD matrix for the current date
    OD_matrix = summarize_district_travel(df_useful, current_date.strftime('%Y-%m-%d'), current_date.strftime('%Y-%m-%d'))
 
    # Save the OD matrix to a CSV file
    file_save_path = fr'C:\Users\User\Downloads\Mobility_traces_new_data_code\OD matrix\district_OD_matrix_{current_date.strftime("%Y-%m-%d")}.csv'
    OD_matrix.to_csv(file_save_path)
    
    # Print the name of the file saved
    print(f'Saved: {file_save_path}')
    
    # Increment the current date
    current_date += timedelta(days=1)

Origin-Destination Matrix:
Travelled to                Champawat District  Dehradun District  \
Travelled from                                                      
Almora District                            1.0                0.0   
Champawat District                         0.0                0.0   
Dehradun District                          0.0                0.0   
Haridwar District                          0.0              155.0   
Nainital District                          0.0                6.0   
Outside                                    2.0              277.0   
Pauri Garhwal District                     0.0               19.0   
Tehri Garhwal District                     0.0               98.0   
Udham Singh Nagar District                 1.0                3.0   
Uttarkashi District                        0.0               22.0   

Travelled to                Haridwar District  Nainital District  Outside  \
Travelled from                                                     

In [13]:
OD_matrix.to_csv(r'C:\Users\User\Downloads\Mobility_traces_new_data_code\OD matrix\District_OD_matrix.csv')

In [1]:
def inter_district_travel(visitors, start_date, end_date):
    """
    Summarizes the travel between districts within a given date range.

    Parameters:
    visitors (pd.DataFrame): The dataframe containing visitor records.
    start_date (str): The start date in the format 'YYYY-MM-DD'.
    end_date (str): The end date in the format 'YYYY-MM-DD'.

    Returns:
    pd.DataFrame: A summary dataframe showing the count of people traveling from one district to another.
    """
    # Ensure datetime column is in datetime format
    visitors['datetime'] = pd.to_datetime(visitors['datetime'], format='%Y-%m-%d %H:%M:%S')

    # Extract date part
    visitors['date'] = visitors['datetime'].dt.date

    # Convert start_date and end_date to datetime objects
    start_date = datetime.strptime(start_date, '%Y-%m-%d').date()
    end_date = datetime.strptime(end_date, '%Y-%m-%d').date()

    # Filter the dataframe based on the date range
    mask = (visitors['date'] >= start_date) & (visitors['date'] <= end_date)
    filtered_visitors = visitors.loc[mask].copy()

    if filtered_visitors.empty:
        print("No records found within the given date range.")
        return pd.DataFrame(columns=['Travelled from', 'Travelled to', 'Count of people'])

    # Shift the district information to compare with the previous row
    # filtered_visitors['prev_district'] = filtered_visitors.groupby('maid')['district_id'].shift(1)
    filtered_visitors['prev_district_name'] = filtered_visitors.groupby('maid')['district'].shift(1)

    # Identify changes in district
    filtered_visitors['changed_district'] = filtered_visitors['district'] != filtered_visitors['prev_district_name']

    # Filter only the rows where district has changed and the previous district is not NaN
    visitors_changes = filtered_visitors[filtered_visitors['changed_district'] & filtered_visitors['prev_district_name'].notna()]

    if visitors_changes.empty:
        print("No district changes found within the given date range.")
        return pd.DataFrame(columns=['Travelled from', 'Travelled to', 'Count of people'])

    # Group by 'Travelled from' and 'Travelled to' and count number of transitions
    travel_summary = visitors_changes.groupby(['prev_district_name', 'district']).size().reset_index(name='Count')

    # Rename columns for clarity
    travel_summary.columns = ['Travelled from', 'Travelled to', 'Count of travels']
    # Create OD matrix
    return travel_summary


In [ ]:
travel_summary_UK = inter_district_travel(df_useful,'2022-09-03', '2022-09-14')
travel_summary_UK

In [25]:
travel_summary_UK.to_csv(r'C:\Users\User\Downloads\Mobility_traces_new_data_code\OD matrix\travel_summary_UK.csv', index=False)

In [32]:
df_3_sep = pd.read_excel(r'C:\Users\User\Downloads\Mobility_traces_new_data_code\all visitors\less_records_for_flowmap.xlsx')
df_3_sep.columns

Index(['maid', 'district', 'latitude', 'longitude', 'datetime', 'location'], dtype='object')

In [33]:
df_3_sep_travel_summary = inter_district_travel(df_3_sep,'2022-09-03', '2022-09-03')
df_3_sep_travel_summary.to_csv(r'C:\Users\User\Downloads\Mobility_traces_new_data_code\OD matrix\less_records_df_3_sep_travel_summary_new_flowmap.csv', index=False)

In [ ]:
import geopandas as gpd
from shapely.geometry import Polygon
from rtree import index

# Assuming poly_file is a GeoDataFrame with polygons
# Create spatial index
sindex = poly_file.sindex

# List to store overlapping pair indices
overlapping_pairs = []

# Iterate through polygons in poly_file
for idx, polygon in poly_file.iterrows():
    # Find possible matches using spatial index
    possible_matches_index = list(sindex.intersection(polygon.geometry.bounds))
    possible_matches = poly_file.iloc[possible_matches_index]
    
    # Iterate through possible matches
    for match_idx, match_polygon in possible_matches.iterrows():
        if polygon.geometry.intersects(match_polygon.geometry) and idx != match_idx:
            overlapping_pairs.append((idx, match_idx))

# Create a set of unique overlapping pairs to avoid duplicates
overlapping_pairs = set(tuple(sorted(pair)) for pair in overlapping_pairs)

# Convert set back to list for printing or further use
overlapping_pairs = list(overlapping_pairs)

# Remove overlapping pairs from poly_file
indices_to_remove = set()
for pair in overlapping_pairs:
    indices_to_remove.update(pair)

# Create a new DataFrame excluding overlapping pairs
poly_file_cleaned = poly_file.drop(index=indices_to_remove)

# Print the number of overlapping pairs of polygons found
print(f'Number of overlapping pairs of polygons: {len(overlapping_pairs)}')

# Optional: Print the actual overlapping pairs for further investigation
print("Overlapping pairs of polygon indices:")
print(overlapping_pairs)

Number of overlapping pairs of polygons: 27
Overlapping pairs of polygon indices:
[(3, 7), (5, 10), (0, 5), (8, 12), (9, 11), (1, 3), (1, 12), (6, 11), (4, 5), (5, 6), (3, 6), (8, 11), (9, 10), (1, 2), (0, 4), (10, 11), (7, 12), (6, 10), (3, 11), (3, 8), (0, 3), (0, 6), (2, 3), (11, 12), (1, 7), (6, 9), (7, 8)]


In [140]:
OD_matrix_2.to_csv(r'C:\Users\User\Downloads\Mobility_traces_new_data_code\OD matrix\OD_matrix_2.csv')

In [36]:
df_useful.shape

(3729555, 7)

In [41]:
# df_useful_with_districts_cleaned

#code to list down people who never travelled
# Identify unique districts visited by each person
unique_districts_per_person = df_useful.groupby('maid')['district'].nunique()
# print(f'unique_districts_per_person: {unique_districts_per_person}')
# Filter out people who visited only one district
people_never_traveled = unique_districts_per_person[unique_districts_per_person == 1].index
# print(f'people_never_traveled: {people_never_traveled}')
# Filter the original dataframe to include only people who never traveled
never_traveled_df = df_useful[df_useful['maid'].isin(people_never_traveled)]
# print(f'never_traveled_df: {never_traveled_df}')
temp_df= never_traveled_df[never_traveled_df['district']=='Outside']
print(f'never_traveled_df for outside people: {temp_df}')
# Count the number of people for each district who never traveled outside
count_never_traveled_per_district = never_traveled_df.groupby('district')['maid'].nunique()

print("Count of people for each district who never traveled outside:")
# print(f'count_never_traveled_per_district: {count_never_traveled_per_district}')
# count_never_traveled_per_district.to_csv(r'C:\Users\User\Downloads\Mobility_traces_new_data_code\OD matrix\Count_of_non_travellers.csv')
# print(f'Save the records at {'/content/drive/MyDrive/Sample_Traces_data/OD matrix/People_stayed_within_districts.csv'}')
# count_never_traveled_per_district.to_csv(r'/content/drive/MyDrive/Sample_Traces_data/OD matrix/People_stayed_within_districts.csv')

never_traveled_df for outside people:                                          maid  latitude  longitude  \
61175    0432358d-e259-4baf-a326-fde5e5769ee2   30.4480    77.6493   
61176    0432358d-e259-4baf-a326-fde5e5769ee2   30.4475    77.6495   
61177    0432358d-e259-4baf-a326-fde5e5769ee2   30.4476    77.6499   
61178    0432358d-e259-4baf-a326-fde5e5769ee2   30.4476    77.6485   
61179    0432358d-e259-4baf-a326-fde5e5769ee2   30.4380    77.6251   
...                                       ...       ...        ...   
3674457  fc06c041-8d89-44aa-8c2d-02d90e90477c   30.6942    77.4288   
3674458  fc06c041-8d89-44aa-8c2d-02d90e90477c   30.6942    77.4285   
3674459  fc06c041-8d89-44aa-8c2d-02d90e90477c   30.6942    77.4285   
3674460  fc06c041-8d89-44aa-8c2d-02d90e90477c   30.6941    77.4285   
3674461  fc06c041-8d89-44aa-8c2d-02d90e90477c   30.6940    77.4285   

                   datetime             state district        date  
61175   2022-09-03 02:15:15  Himachal Pradesh  Outsi

In [45]:
# df_useful_with_districts_cleaned
merged_data = pd.read_csv(r"C:\Users\User\Downloads\Mobility_traces_new_data_code\all visitors\visitors_with_all_states_and_districts.csv")
print(f'merged_data shape : {merged_data.shape}')
#code to list down people who never travelled
# Identify unique districts visited by each person
unique_districts_per_person = merged_data.groupby('maid')['district'].nunique()
# print(f'unique_districts_per_person: {unique_districts_per_person}')
# Filter out people who visited only one district
people_never_traveled = unique_districts_per_person[unique_districts_per_person == 1].index
# print(f'people_never_traveled: {people_never_traveled}')
# Filter the original dataframe to include only people who never traveled
never_traveled_df = merged_data[merged_data['maid'].isin(people_never_traveled)]
# print(f'never_traveled_df: {never_traveled_df}')
temp_df= never_traveled_df[never_traveled_df['district']=='Outside']
print(f'never_traveled_df for outside people: {temp_df}')
# Count the number of people for each district who never traveled outside
count_never_traveled_per_district = never_traveled_df.groupby('district')['maid'].nunique()

print("Count of people for each district who never traveled outside:")
# print(f'count_never_traveled_per_district: {count_never_traveled_per_district}')
# count_never_traveled_per_district.to_csv(r'C:\Users\User\Downloads\Mobility_traces_new_data_code\OD matrix\Count_of_non_travellers.csv')
# print(f'Save the records at {'/content/drive/MyDrive/Sample_Traces_data/OD matrix/People_stayed_within_districts.csv'}')
# count_never_traveled_per_district.to_csv(r'/content/drive/MyDrive/Sample_Traces_data/OD matrix/People_stayed_within_districts.csv')

merged_data shape : (3729555, 11)
never_traveled_df for outside people:                                          maid             datetime  latitude  \
61175    0432358d-e259-4baf-a326-fde5e5769ee2  2022-09-03 02:15:15   30.4480   
61176    0432358d-e259-4baf-a326-fde5e5769ee2  2022-09-03 03:48:12   30.4475   
61177    0432358d-e259-4baf-a326-fde5e5769ee2  2022-09-03 07:46:04   30.4476   
61178    0432358d-e259-4baf-a326-fde5e5769ee2  2022-09-03 08:42:45   30.4476   
61179    0432358d-e259-4baf-a326-fde5e5769ee2  2022-09-03 09:45:30   30.4380   
...                                       ...                  ...       ...   
3674457  fc06c041-8d89-44aa-8c2d-02d90e90477c  2022-09-14 10:17:14   30.6942   
3674458  fc06c041-8d89-44aa-8c2d-02d90e90477c  2022-09-14 12:22:14   30.6942   
3674459  fc06c041-8d89-44aa-8c2d-02d90e90477c  2022-09-14 15:05:45   30.6942   
3674460  fc06c041-8d89-44aa-8c2d-02d90e90477c  2022-09-14 17:33:12   30.6941   
3674461  fc06c041-8d89-44aa-8c2d-02d90e90477c  2

In [46]:
temp_df.to_csv(r'C:\Users\User\Downloads\Mobility_traces_new_data_code\OD matrix\People_stayed_outside.csv', index=False)
temp_df['maid'].nunique()
grouped = temp_df.groupby('maid')
for maid, group in grouped:
    group.to_csv(fr'C:\Users\User\Downloads\Mobility_traces_new_data_code\OD matrix\People_stayed_outside\stayed_outside_{maid}.csv', index=False)


In [16]:
count_never_traveled_per_district.to_csv(r'C:\Users\User\Downloads\Mobility_traces_new_data_code\OD matrix\Count_of_non_travellers.csv')